In [ ]:
from functools import reduce
# Data awal (diinisialisasi di luar fungsi bisnis)
accounts = {}  # Key: NIM, Value: {"password": pass, "role": role}
personnel_data = []  # List of Dictionaries: {"nim": str, "name": str, "rank": str, "unit_id": int}
unit_data = []  # {"unit_id": int, "unit_name": str}
mission_data = []  # (mission_id, mission_name, location, [involved_personnel])

# ======================
# Fungsi (Pure Functions)
# ======================

def register(accounts, nim, password, role):
    # Menggunakan ternary untuk mengurangi if-else dan tetap mengembalikan tuple dengan pesan.
    return (
        accounts if nim in accounts else {**accounts, nim: {"password": password, "role": role}},
        "NIM already registered!" if nim in accounts else "Account successfully registered!"
    )

def add_profile(personnel_data, nim, name, rank, unit_id, unit_data):
    # Menggunakan any() untuk pengecekan NIM dan unit_id dengan lebih ringkas.
    if any(p["nim"] == nim for p in personnel_data):
        return personnel_data, f"Profile for NIM {nim} already exists."

    if unit_id != 0 and not any(u["unit_id"] == unit_id for u in unit_data):
        return personnel_data, f"Unit ID {unit_id} not found. Profile not saved."

    # List comprehension untuk membuat profil baru dan menggabungkannya.
    return personnel_data + [{"nim": nim, "name": name, "rank": rank, "unit_id": unit_id}], \
           f"Profile for NIM {nim} has been added."

def login(accounts, nim, password):
    # Menggabungkan pengecekan NIM dan password dalam satu blok ternary.
    return (
        (nim, f"Welcome, {nim}!") if nim in accounts and accounts[nim]["password"] == password else
        (None, "Incorrect password." if nim in accounts else "NIM not found. Please register first.")
    )

def view_profile(personnel_data, unit_data, nim):
    # Menggunakan filter() untuk mencari profil dengan NIM yang cocok.
    profile = next(filter(lambda p: p["nim"] == nim, personnel_data), None)
    if not profile:
        return f"No profile found for NIM {nim}. Please update your profile."

    # Menggunakan generator expression untuk menemukan nama unit.
    unit_name = next((u["unit_name"] for u in unit_data if u["unit_id"] == profile["unit_id"]), "Unknown")
    return f"Name: {profile['name']}, Rank: {profile['rank']}, Unit: {unit_name}"

def update_profile(personnel_data, nim, new_name, new_rank, new_unit_id, unit_data):
    # Pengecekan unit_id langsung dengan any().
    if new_unit_id and not any(u["unit_id"] == new_unit_id for u in unit_data):
        return personnel_data, f"Unit ID {new_unit_id} not found. Update cancelled."

    # Menggunakan fungsi map() untuk memperbarui data secara fungsional.
    def update(p):
        return {
            "nim": p["nim"],
            "name": new_name or p["name"],
            "rank": new_rank or p["rank"],
            "unit_id": new_unit_id if new_unit_id is not None else p["unit_id"]
        } if p["nim"] == nim else p

    # Map mengembalikan iterator, jadi dikonversi ke list.
    updated_data = list(map(update, personnel_data))
    return (
        updated_data, "Profile updated successfully." if updated_data != personnel_data else
        (personnel_data, f"No profile found for NIM {nim}.")
    )

def delete_profile(personnel_data, accounts, nim):
    # List comprehension untuk memfilter data profil dan akun.
    return (
        [p for p in personnel_data if p["nim"] != nim],
        {k: v for k, v in accounts.items() if k != nim},
        f"Profile and account for NIM {nim} deleted successfully."
    )

def add_unit(unit_data, unit_id, unit_name):
    # Ternary untuk mencegah duplikasi unit_id dan menambahkan unit baru jika belum ada.
    return (
        unit_data if any(u["unit_id"] == unit_id for u in unit_data) else 
        unit_data + [{"unit_id": unit_id, "unit_name": unit_name}],
        f"Unit ID {unit_id} already exists." if any(u["unit_id"] == unit_id for u in unit_data) else 
        f"Unit {unit_name} added successfully."
    )

def add_mission(mission_data, mission_id, mission_name, location, involved_personnel):
    # Langsung menambahkan misi dengan list + tuple baru.
    return (
        mission_data + [(mission_id, mission_name, location, involved_personnel)],
        f"Mission {mission_name} added successfully."
    )

def delete_unit(unit_data, unit_id):
    # List comprehension untuk menghapus unit berdasarkan unit_id.
    return (
        [u for u in unit_data if u["unit_id"] != unit_id],
        f"Unit ID {unit_id} deleted successfully."
    )

def delete_mission(mission_data, mission_id):
    # List comprehension untuk menghapus misi berdasarkan mission_id.
    return (
        [m for m in mission_data if m[0] != mission_id],
        f"Mission ID {mission_id} deleted successfully."
    )

# ======================
# Wrapper Fungsi I/O
# ======================

def register_wrapper(accounts):
    # Menggabungkan input dalam satu baris untuk lebih ringkas.
    nim, password, role = input("Enter your NIM: "), input("Enter your password: "), input("Enter your role (admin/user): ")
    new_accounts, message = register(accounts, nim, password, role)
    print(message)
    return new_accounts

def add_profile_wrapper(personnel_data, unit_data, nim):
    # Menangkap input dengan lebih ringkas.
    name, rank = input("Enter your name: "), input("Enter your rank: ")
    try:
        unit_id = int(input("Enter your Unit ID (0 if unknown): "))
    except ValueError:
        print("Invalid Unit ID.")
        return personnel_data
    new_personnel_data, message = add_profile(personnel_data, nim, name, rank, unit_id, unit_data)
    print(message)
    return new_personnel_data

def login_wrapper(accounts):
    # Menggabungkan input dalam satu langkah.
    nim, password = input("Enter your NIM: "), input("Enter your password: ")
    nim, message = login(accounts, nim, password)
    print(message)
    return nim

def update_profile_wrapper(personnel_data, unit_data, nim):
    # Memastikan input None jika kosong.
    new_name, new_rank = input("Enter new name (leave blank to keep current): ") or None, input("Enter new rank (leave blank to keep current): ") or None
    new_unit_id = input("Enter new Unit ID (leave blank to keep current): ") or None
    if new_unit_id:
        try:
            new_unit_id = int(new_unit_id)
        except ValueError:
            print("Invalid Unit ID.")
            return personnel_data
    new_data, message = update_profile(personnel_data, nim, new_name, new_rank, new_unit_id, unit_data)
    print(message)
    return new_data

# ======================
# Menu dan Main Loop
# ======================

def user_menu(nim, accounts, personnel_data, unit_data):
    while True:
        # Memperpendek opsi menu dengan langsung menggunakan input().
        choice = input("\nUser Menu:\n1. View Profile\n2. Update Profile\n3. Delete Profile\n0. Logout\nChoose an option: ")
        if choice == "1":
            print(view_profile(personnel_data, unit_data, nim))
        elif choice == "2":
            personnel_data = update_profile_wrapper(personnel_data, unit_data, nim)
        elif choice == "3" and input("Are you sure you want to delete your profile? (yes/no): ").lower() == 'yes':
            personnel_data, accounts, message = delete_profile(personnel_data, accounts, nim)
            print(message)
            break
        elif choice == "0":
            break
        else:
            print("Invalid choice.")

def main():
    global accounts, personnel_data, unit_data

    while True:
        # Mempercepat input dengan menggabungkan opsi menu.
        choice = input("\nMain Menu:\n1. Register\n2. Login\n0. Exit\nChoose an option: ")
        if choice == "1":
            accounts = register_wrapper(accounts)
        elif choice == "2":
            nim = login_wrapper(accounts)
            if nim:
                user_menu(nim, accounts, personnel_data, unit_data)
        elif choice == "0":
            break
        else:
            print("Invalid choice.")

main()



Main Menu:
1. Register
2. Login
0. Exit
Choose an option:  1
Enter your NIM:  12
Enter your password:  12
Enter your role (admin/user):  user


Account successfully registered!



Main Menu:
1. Register
2. Login
0. Exit
Choose an option:  2
Enter your NIM:  12
Enter your password:  12


Welcome, 12!



User Menu:
1. View Profile
2. Update Profile
3. Delete Profile
0. Logout
Choose an option:  1


No profile found for NIM 12. Please update your profile.



User Menu:
1. View Profile
2. Update Profile
3. Delete Profile
0. Logout
Choose an option:  3
Are you sure you want to delete your profile? (yes/no):  y


Invalid choice.



User Menu:
1. View Profile
2. Update Profile
3. Delete Profile
0. Logout
Choose an option:  3
Are you sure you want to delete your profile? (yes/no):  yes


Profile and account for NIM 12 deleted successfully.
